In [ ]:
using Pkg
Pkg.add("Primes")
using Primes
using BenchmarkTools
using Dates
Pkg.add("Format")
using Format

function fast_sigma1(n)::Int
    result = 1
    for (p, e) in eachfactor(n)
        result *= div(p^(e+1) - 1, (p - 1))
    end
    return result
end

function is_perfect_square(num)
    # Check if the number is positive
    num <= 0 && return false

    # Calculate the square root of the number
    square_root = isqrt(num)
   
    # Check if the square of the square root is equal to the original number
    return (square_root^2 == num) || ((square_root+1)^2 == num)
end

function divide_until_odd(n)
    while n % 2 == 0 && n != 0
        n = n ÷ 2
    end
    return n
end

function is_largest_odd_divisor_square(n)
    return is_perfect_square(divide_until_odd(n))
end

function nearsuperperfectchecker(range)
    outputs = Vector{Vector{Int}}()
    for n in range
        if n % 16 == 8
            continue
        end
        if n % 10 == 0
            n % 25 > 0 && continue
        end
        if n % 6 == 0
            !is_largest_odd_divisor_square(n) && continue
        end
        s1 = fast_sigma1(n)
        s1 > 3n && continue
        if n % 2 == 1
            !is_largest_odd_divisor_square(s1) && continue
        end
        s = fast_sigma1(s1)
        if 2n != s
            if n % (s - 2n) == 0
                push!(outputs, [n, s, s - 2n])
            end
        end
    end
    # if length(outputs) > 0
        # println(outputs)
    # end
    return outputs
end

In [ ]:
@btime nearsuperperfectchecker(1:1_000_000)

In [ ]:
nums_per_task = 2^16
tasks_per_chunk = 2^10
num_chunks = 2^12

total_nums = num_chunks * tasks_per_chunk * nums_per_task
format(total_nums, commas=true)

In [ ]:

function task_range(chunk_index, task_index) 
    previous_chunk_stop = (chunk_index - 1) * tasks_per_chunk * nums_per_task
    task_start = previous_chunk_stop + ((task_index - 1) * nums_per_task) + 1
    task_stop = task_start + nums_per_task - 1
    return task_start:task_stop
end
all_outputs = Vector{Vector{Int}}[]

@time for chunk_index in 1:num_chunks
    chunk_stop = chunk_index * tasks_per_chunk * nums_per_task
    chunk_outputs = [Threads.@spawn nearsuperperfectchecker(task_range(chunk_index, task_index)) for task_index in 1:tasks_per_chunk]
    chunk_output = vcat(fetch.(chunk_outputs)...)
    println("$(now()) Output for chunk $chunk_index until $(format(chunk_stop, commas=true)): $chunk_output")
    flush(stdout)
    push!(all_outputs, chunk_output)
end